In [ ]:
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
import keras.layers as layers
from pathlib import Path
import matplotlib.pyplot as plt
import os

# Custom Metric to load the models without problems
import keras.backend as K
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
def kullback_leibler_divergence_test_2(y_true, y_pred):

    y_true = K.clip(y_true, K.epsilon(), 1)
    y_pred = K.clip(y_pred, K.epsilon(), 1)
    return K.sum(y_true * abs(K.log(y_true /  y_pred)), axis=-1)

custom_metr = custom_objects={
        "kullback_leibler_divergence_test_2":kullback_leibler_divergence_test_2,
        "dice_coef": dice_coef}

# def create_path_from_title(title):
#     path = title+"/"+title+".h5"
#     return path

# Loadcases chosen : LoadCase_7885,LoadCase_7818, LoadCase_7831, LoadCase_7852
def save_fig(fig_id,path, tight_layout=True, fig_extension="png", resolution=300):
    os.makedirs(path,exist_ok=True)
    path = os.path.join(path, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
# Define The Path of the Test Set
# Make a function to load the Inputs from the Files as np.array to predict from it
def create_instance(TEST_SET_PATH):
    instances = []
    input_path =os.path.join(TEST_SET_PATH,"Input")
    input_names = os.listdir(input_path)
    for i in input_names:
        loaded_array_x = np.load(os.path.join(input_path,i),allow_pickle = True)
        instances.append(loaded_array_x)
    return np.array(instances),input_names
# Flipping the Image so we have Cartesian coordinates
def flip_for_imshow(structure):
    get_shape = np.shape(structure)[1:]
    print(get_shape)

    structure = (np.reshape(structure,get_shape))
    return structure
# Gets the Volume Fraction of given Structure
def volfrac_of_structure(structure):
    get_shape = np.shape(structure)
    print(get_shape)
    sum_ = np.sum(np.reshape(structure,get_shape))
    
    div_ = sum_/(int(get_shape[0])*int(get_shape[1]))
    return div_
# Imports the Ground Truth
def get_ground_truth(TEST_SET_PATH):
    instances = []
    output_path =os.path.join(TEST_SET_PATH,"Output")
    output_names = os.listdir(output_path)
    for i in output_names:
        loaded_array_x = np.load(os.path.join(output_path,i),allow_pickle = True)
        instances.append(loaded_array_x)
    return np.array(instances)

# Calling this function predicts the given instances
def create_predictions(model,test_instances):
    predictions = model.predict(test_instances)
    return predictions
# Loads the Models and puts them into a list
def load_models(IDs,TRIALS_ROOT):
    loaded_models = []
    for ID in IDs:
        model_path  =os.path.join(TRIALS_ROOT,ID,str(ID+".h5"))
        loaded_model = tf.keras.models.load_model(model_path,custom_metr)
        loaded_models.append(loaded_model)
    return loaded_models


In [ ]:
# Root of the Model Trials
TRIALS_ROOT = ""
# Trial Ids
IDs = [""]
# Path to save images to
IMG_PATH = ""
# IMG title
comparison_title = ""
# Path of the data the model predicts from
TEST_SET_PATH = ""
# Choosen Loadcases to compare
loadcase_names = [""]


In [ ]:
tf.keras.backend.clear_session()
loaded_models = load_models(IDs,TRIALS_ROOT)

In [ ]:
# This function compares the different predictions with the Ground Truth based on a loadcase

def create_prediction_images(loaded_models,loadcase_name,TEST_SET_PATH,IDs,comparison_title,
                             IMG_PATH):
# Create the Test set
    test_instances,test_names = create_instance(TEST_SET_PATH)
# Get the Index of given Loadcase
    loadcase_index = test_names.index(loadcase_name)
# Get the Ground truth
    ground_truth = get_ground_truth(TEST_SET_PATH)
    model_predictions = []
# Needed for the amount of Images shown
    len_IDs = len(IDs)
# Title under which the Image is saved
    img_title = str("Loadcase "+str(loadcase_name)+str(comparison_title))
# Predict with each model and append those predictions
    for model in loaded_models:
        predictions = create_predictions(model,test_instances)
        model_predictions.append(predictions)
# Get the one Datapoint for the Loadcase
    ground_truth_loadcase = flip_for_imshow(ground_truth[loadcase_index])
# Define the Plot
    f = plt.figure(figsize = (20,20))
# First axis and plot is the Ground truth
    ax  = f.add_subplot(len_IDs+1, 1, 1)
    ax.set_title(str("Ground Truth "+"has the Volume Fraction "+str(volfrac_of_structure(
    ground_truth_loadcase))))
    plt.imshow(ground_truth_loadcase, cmap="gray_r")
# For each of the Models an Subplot is generated
    for i,s in enumerate(model_predictions):
        ax = f.add_subplot( len_IDs+1, 1, i + 2)
        predicted_loadcase_structure = flip_for_imshow(s[loadcase_index])
        ax.set_title(str("Model: "+str(IDs[i])+" has the Volume Fraction "+str(volfrac_of_structure(
        predicted_loadcase_structure))))
        plt.imshow(predicted_loadcase_structure, cmap="gray_r")
# Save the Image
    save_fig(img_title,IMG_PATH)
    plt.show(block=True)





In [ ]:
# We got multiple test Loadcases
for i in loadcase_names:
    create_prediction_images(loaded_models,i,TEST_SET_PATH,IDs,comparison_title,
                             IMG_PATH)
